
## Implementación y resolución como problema cuádratico de la formulación dual SVM con holgura y kernel.
**Integrantes**
> Mario Mallea, Maximiliano Ramírez y Hugo Rocha.

**Descripción**
> En este cuaderno resolveremos el problema de clasificación via solver cvxopt el problema dual tipo SVM.

**Herramientas**
> Python version 3.7.9; version 1.2.5 de cvxopt 



In [1]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)


Python version
3.7.9 | packaged by conda-forge | (default, Dec  9 2020, 20:36:16) [MSC v.1916 64 bit (AMD64)]
Version info.
sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)


version 1.2.5 de cvxopt

In [2]:
pip install cvxopt

Note: you may need to restart the kernel to use updated packages.


Cargamos las librerias a utilizar:

In [73]:
import numpy as np
import random
import pandas as pd
from cvxopt import matrix, solvers
from sklearn.datasets import load_breast_cancer
from datetime import datetime
from sklearn.model_selection import train_test_split


Cargamos los datos del cancer mamario

In [81]:
dataframe = load_breast_cancer()
df = pd.DataFrame(np.c_[dataframe['data'], dataframe['target']],
                  columns= np.append(dataframe['feature_names'], ['target']))

data = df.values
ix = [i for i in range(data.shape[1]) if i != df.shape[1]-1]
X, y = data[:, ix], data[:, df.shape[1]-1]




for i in range(y.shape[0]): ## transformamos las etiquetas a clasificar 
    if y[i]==0: 
        y[i]=-1 #benigno es -1, maligo se queda con 1
        
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Definimos diferentes kernels

In [101]:
def poly_kernel(x, z, degree, intercept):
        return np.power(np.matmul(x, z.T) + intercept, degree)

def gaussian_kernel(x, z,sigma):
    n = x.shape[0]
    m = z.shape[0]
    xx = np.dot(np.sum(np.power(x, 2), 1).reshape(n, 1), np.ones((1, m)))
    zz = np.dot(np.sum(np.power(z, 2), 1).reshape(m, 1), np.ones((1, n)))     
    return np.exp(-(xx + zz.T - 2 * np.dot(x, z.T)) / (2 * sigma ** 2))

def linear_kernel(x, z):
    return np.matmul(x, z.T)

Establecemos el problema como uno progrmación cuadrática y resolvemos con solver obteniendo los multiplicadores de Lagrange:

In [120]:
import numpy as np
import cvxopt
def fit(X, y, kernel, C):
    n_samples, n_features = X.shape
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            K[i,j] = kernel(X[i], X[j])
            
    P = cvxopt.matrix(np.outer(y,y) * K)
    q = cvxopt.matrix(np.ones(n_samples) * -1)
    A = cvxopt.matrix(y, (1,n_samples))
    b = cvxopt.matrix(0.0)
    if C is None:      # hard-margin SVM
       G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
       h = cvxopt.matrix(np.zeros(n_samples))
    else:              # soft-margin SVM
       G = cvxopt.matrix(np.vstack((np.diag(np.ones(n_samples) * -1), np.identity(n_samples))))
       h = cvxopt.matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))
    # Resuelve problema cuadrático QP
    solution = cvxopt.solvers.qp(P, q, G, h, A, b)
    print(solution)
    # multiplicadores
    a = np.ravel(solution['x'])
    # vectores de soporte con multiplicadores de lagrange no nulos
    
    return a
    

Establecemos un criterio de convergencia y predecimos

In [121]:
a= fit(X_train, y_train, linear_kernel, 100)
ind = (a > 1e-4).flatten()
sv = X_train[ind]
sv_y = y_train[ind]
alphas = a[ind]



     pcost       dcost       gap    pres   dres
 0:  5.0903e+03 -2.1362e+06  5e+06  6e-01  4e-07
 1:  9.9345e+03 -8.1552e+05  1e+06  2e-01  3e-07
 2:  8.7265e+03 -4.3656e+05  7e+05  7e-02  2e-07
 3:  4.6473e+03 -1.6971e+05  2e+05  2e-02  1e-07
 4:  2.1750e+03 -6.9113e+04  9e+04  5e-03  1e-07
 5: -3.4831e+01 -1.9138e+04  2e+04  7e-04  1e-07
 6: -8.9233e+02 -6.8594e+03  6e+03  1e-04  1e-07
 7: -1.2031e+03 -4.8062e+03  4e+03  4e-05  1e-07
 8: -1.3424e+03 -4.7165e+03  3e+03  4e-05  1e-07
 9: -1.3986e+03 -4.1099e+03  3e+03  1e-05  2e-07
10: -1.6384e+03 -3.2267e+03  2e+03  6e-06  1e-07
11: -1.6918e+03 -2.7392e+03  1e+03  2e-06  2e-07
12: -1.8038e+03 -2.5178e+03  7e+02  8e-07  2e-07
13: -1.9033e+03 -2.1932e+03  3e+02  8e-14  2e-07
14: -1.9623e+03 -2.0995e+03  1e+02  7e-14  2e-07
15: -2.0125e+03 -2.0181e+03  6e+00  6e-14  2e-07
16: -2.0146e+03 -2.0147e+03  1e-01  3e-13  2e-07
17: -2.0146e+03 -2.0146e+03  1e-03  2e-13  2e-07
18: -2.0146e+03 -2.0146e+03  1e-05  8e-14  2e-07
19: -2.0146e+03 -2.01

In [122]:
b = sv_y - np.sum(linear_kernel(sv, sv) * alphas * sv_y, axis=0)
b = np.sum(b) / b.size

predictions=[]
for i in range(X_test.shape[0]):
    prod=np.sum(linear_kernel(sv, X_test[i,:]).T * alphas * sv_y, axis=0) + b
    predictions.append(np.sign(prod))
    
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[ 62,   1],
       [  4, 104]], dtype=int64)

Fabricamos Data artificial con 2500 nuevos casos.

In [25]:
df_copiado=df.copy()

In [32]:
%%time
n_datos= 2500

df_maligno = df_copiado[df_copiado["target"]==1]
for i in range(n_datos):
    l=[]
    for j in df_copiado.columns[:-1]:
        l.append(random.uniform(df_maligno[j].min(), df_maligno[j].max()))
    df.loc[i+569,:]= l+[1]

Wall time: 14.9 s


In [34]:
%%time
n_datos= 2500
n_datos_original_maligno=df.shape[0]
df_benigno = df_copiado[df_copiado["target"]==-1]
for i in range(n_datos):
    l=[]
    for j in df_copiado.columns[:-1]:
        l.append(random.uniform(df_benigno[j].min(), df_benigno[j].max()))
    df.loc[i+n_datos_original_maligno,:]= l+[-1]

Wall time: 14.5 s


In [36]:
df.to_csv(r'C:\Users\Mario\python\machine_learning\cancer_artificial_equilibrado.csv', index = False)

In [38]:
big_data= pd.read_csv("cancer_artificial_equilibrado.csv")

In [50]:
data = big_data.values
ix = [i for i in range(data.shape[1]) if i != big_data.shape[1]-1]
X, y = data[150:, ix], data[150:, big_data.shape[1]-1]


In [52]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Inicio =", current_time)

a= fit(X, y, linear_kernel, 0.5)
ind = (a > 1e-4).flatten()
sv = X[ind]
sv_y = y[ind]
alphas = a[ind]

#Calcular bias
b = sv_y - np.sum(linear_kernel(sv, sv) * alphas * sv_y, axis=0)
b = np.sum(b) / b.size

prod = np.sum(linear_kernel(sv, X).T * alphas * sv_y, axis=0) + b
predictions = np.sign(prod)

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Final =", current_time)


Inicio = 17:25:58
     pcost       dcost       gap    pres   dres
 0: -4.0250e+02 -5.8252e+03  6e+04  6e+00  2e-07
 1: -1.9854e+02 -4.2280e+03  1e+04  8e-01  2e-07
 2: -1.0914e+02 -1.3555e+03  2e+03  1e-01  1e-07
 3: -7.4691e+01 -6.5493e+02  9e+02  6e-02  6e-08
 4: -6.1814e+01 -4.5545e+02  6e+02  4e-02  5e-08
 5: -5.4682e+01 -3.4878e+02  5e+02  3e-02  4e-08
 6: -4.8808e+01 -2.7485e+02  4e+02  2e-02  4e-08
 7: -4.4812e+01 -2.1543e+02  3e+02  1e-02  3e-08
 8: -4.2240e+01 -1.7313e+02  2e+02  9e-03  3e-08
 9: -4.0091e+01 -1.4447e+02  2e+02  7e-03  3e-08
10: -3.8860e+01 -1.2579e+02  1e+02  5e-03  2e-08
11: -3.7688e+01 -1.1072e+02  1e+02  4e-03  2e-08
12: -3.6912e+01 -8.3087e+01  7e+01  2e-03  3e-08
13: -3.6420e+01 -6.9239e+01  5e+01  1e-03  2e-08
14: -3.6133e+01 -5.2326e+01  2e+01  3e-04  3e-08
15: -3.5765e+01 -5.0133e+01  2e+01  2e-04  3e-08
16: -3.5943e+01 -4.9157e+01  2e+01  2e-04  2e-08
17: -3.5753e+01 -4.8364e+01  1e+01  9e-05  2e-08
18: -3.6898e+01 -4.5493e+01  9e+00  5e-05  3e-08
19:

In [62]:
b = sv_y - np.sum(linear_kernel(sv, sv) * alphas * sv_y, axis=0)
b = np.sum(b) / b.size

predictions=[]
for i in range(150):
    prod=np.sum(linear_kernel(sv, X[i,:]).T * alphas * sv_y, axis=0) + b
    predictions.append(np.sign(prod))
    
from sklearn.metrics import confusion_matrix
y_true = data[:150, big_data.shape[1]-1]
confusion_matrix(y_true, predictions)

array([[28, 55],
       [25, 42]], dtype=int64)